In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import yaml
from collections import defaultdict
import pickle as pkl
import copy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import respy as rp
from respy.pre_processing.model_processing import process_params_and_options

from python.auxiliary_setup import *
from python.auxiliary_weighting import *
from python.auxiliary_plots import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Observed-Data" data-toc-modified-id="Observed-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Observed Data</a></span></li><li><span><a href="#Data-Moments" data-toc-modified-id="Data-Moments-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data Moments</a></span></li><li><span><a href="#Weighting-Matrix" data-toc-modified-id="Weighting-Matrix-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Weighting Matrix</a></span></li><li><span><a href="#Criterion-Function" data-toc-modified-id="Criterion-Function-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Criterion Function</a></span></li><li><span><a href="#Estimation-Exercise" data-toc-modified-id="Estimation-Exercise-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Estimation Exercise</a></span><ul class="toc-item"><li><span><a href="#Estimation-for-one-parameter" data-toc-modified-id="Estimation-for-one-parameter-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Estimation for one parameter</a></span><ul class="toc-item"><li><span><a href="#Simulated-Moments" data-toc-modified-id="Simulated-Moments-6.1.1"><span class="toc-item-num">6.1.1&nbsp;&nbsp;</span>Simulated Moments</a></span></li><li><span><a href="#Optimization-Procedure" data-toc-modified-id="Optimization-Procedure-6.1.2"><span class="toc-item-num">6.1.2&nbsp;&nbsp;</span>Optimization Procedure</a></span><ul class="toc-item"><li><span><a href="#Upper-and-Lower-Bounds" data-toc-modified-id="Upper-and-Lower-Bounds-6.1.2.1"><span class="toc-item-num">6.1.2.1&nbsp;&nbsp;</span>Upper and Lower Bounds</a></span></li><li><span><a href="#Constraints" data-toc-modified-id="Constraints-6.1.2.2"><span class="toc-item-num">6.1.2.2&nbsp;&nbsp;</span>Constraints</a></span></li><li><span><a href="#Optimize" data-toc-modified-id="Optimize-6.1.2.3"><span class="toc-item-num">6.1.2.3&nbsp;&nbsp;</span>Optimize</a></span></li></ul></li></ul></li><li><span><a href="#Chatter-in-the-Criterion-Function" data-toc-modified-id="Chatter-in-the-Criterion-Function-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Chatter in the Criterion Function</a></span><ul class="toc-item"><li><span><a href="#Changing-the-Simulation-Seed" data-toc-modified-id="Changing-the-Simulation-Seed-6.2.1"><span class="toc-item-num">6.2.1&nbsp;&nbsp;</span>Changing the Simulation Seed</a></span></li><li><span><a href="#Multiple-Simulation-Seeds" data-toc-modified-id="Multiple-Simulation-Seeds-6.2.2"><span class="toc-item-num">6.2.2&nbsp;&nbsp;</span>Multiple Simulation Seeds</a></span></li><li><span><a href="#Changing-the-Simulation-Seed-for-increasing-Sample-Size-of-Simulated-Agents" data-toc-modified-id="Changing-the-Simulation-Seed-for-increasing-Sample-Size-of-Simulated-Agents-6.2.3"><span class="toc-item-num">6.2.3&nbsp;&nbsp;</span>Changing the Simulation Seed for increasing Sample Size of Simulated Agents</a></span></li><li><span><a href="#Changing-the-Simulation-Seed-for-increasing-Sample-Size-of-Simulated-and-Observed-Agents" data-toc-modified-id="Changing-the-Simulation-Seed-for-increasing-Sample-Size-of-Simulated-and-Observed-Agents-6.2.4"><span class="toc-item-num">6.2.4&nbsp;&nbsp;</span>Changing the Simulation Seed for increasing Sample Size of Simulated <em>and</em> Observed Agents</a></span></li></ul></li><li><span><a href="#Moving-away-from-the-Optimum" data-toc-modified-id="Moving-away-from-the-Optimum-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Moving away from the Optimum</a></span><ul class="toc-item"><li><span><a href="#Retrieving-the-true-Parameter-Vector" data-toc-modified-id="Retrieving-the-true-Parameter-Vector-6.3.1"><span class="toc-item-num">6.3.1&nbsp;&nbsp;</span>Retrieving the true Parameter Vector</a></span></li></ul></li><li><span><a href="#Derivative-Based-Optimization-Algorithm" data-toc-modified-id="Derivative-Based-Optimization-Algorithm-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Derivative-Based Optimization Algorithm</a></span></li></ul></li><li><span><a href="#References" data-toc-modified-id="References-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>References</a></span></li></ul></div>

## Introduction

The simulated method of moments approach to estimating model parameters is to minimize a certain distance between observed moments and simulated moments with respect to the parameters that generate the simulated model.

Denote $X$ our observed data and $m(X)$ the vector of observed moments. To construct the criterion function, we use the parameter vector $\theta$ to simulate data from the  model $\hat{X}$. We can then calculate the simulated moments $m(\hat{X}| \theta)$.

The criterion function is then given by 

\begin{equation}
\psi(\theta) = (m(X) - m(\hat{X}| \theta))'\Omega(m(X) - m(\hat{X}| \theta))
\end{equation}

where the difference between observed and simulated moments $(m(X) - m(\hat{X}| \theta))$ constitutes a vector of the dimension $1\times M$ with $1,..,M$ denoting the number of moments. The $M\times M$ weighting matrix is given by $\Omega$. 

The SMM estimator is defined as the solution to 

\begin{equation}
\hat{\theta}(\Omega) = \underset{\theta}{\operatorname{argmin}} \psi(\theta).
\end{equation}

The criterion function is thus a strictly positive scalar and the estimator depends on the choice of moments $m$ and the weighting matrix $\Omega$. The weighting matrix applies some scaling for discrepancies between the observed and simulated moments. If we use the identity matrix, each moment is given equal weight and the criterion function reduces to the sum of squared moment deviations. 

Aside from the choice of moments and weighting matrix, some other important choices that influence the the estimation are the simulator itself and the algorithm and specifications for the optimization procedure. Many explanations for simulated method of moments estimation also feature the number of simulations as a factor that is to be determined for estimation. We can ignore this factor for now since we are working with a large simulated dataset.

In the following we will set up the data, moments and weighting matrix needed to construct the criterion function and subsequently try to estimating the parameters of the model using this SMM setup.

##  Observed Data

We generate our model and data using [respy](https://respy.readthedocs.io/en/latest/) and a simple Robinson Crusoe model. In this model, the agent, Robinson Crusoe, in each time period decides between two choice options: working (i.e. going fishing) or spending time in the hammock. 

We can use respy to simulate the data for this exercise.

In [2]:
params_true, options = load_model_specs()

In [3]:
# Generate observed data from model.
simulate = rp.get_simulate_func(params_true, options)
data_obs = simulate(params_true)

Let's take a look at the model specifications.

In [4]:
params_true 

value
category       name                       
delta          delta                 0.950
wage_fishing   exp_fishing           0.070
nonpec_fishing constant             -0.100
nonpec_hammock constant              1.046
shocks_sdcorr  sd_fishing            0.010
               sd_hammock            0.010
               corr_hammock_fishing  0.000

In [5]:
options

{'estimation_draws': 100,
 'estimation_seed': 100,
 'estimation_tau': 0.001,
 'interpolation_points': -1,
 'n_periods': 5,
 'simulation_agents': 1000,
 'simulation_seed': 132,
 'solution_draws': 100,
 'solution_seed': 456,
 'covariates': {'constant': '1'}}

In [6]:
data_obs.head(10)

,Identifier,Period,Choice,Wage,Experience_Fishing,Type,Nonpecuniary_Reward_Fishing,Nonpecuniary_Reward_Hammock,Wage_Fishing,Flow_Utility_Fishing,Flow_Utility_Hammock,Value_Function_Fishing,Value_Function_Hammock,Shock_Reward_Fishing,Shock_Reward_Hammock,Discount_Rate
0,0,0,fishing,1.007197,0,0,-0.1,1.046,1.000000,0.907197,1.056305,4.747606,4.742931,1.007197,0.010305,0.95
1,0,1,fishing,1.082581,1,0,-0.1,1.046,1.072508,0.982581,1.046042,4.052627,3.892530,1.009392,0.000042,0.95
2,0,2,fishing,1.163836,2,0,-0.1,1.046,1.150274,1.063836,1.053231,3.245041,3.074449,1.011790,0.007231,0.95
3,0,3,fishing,1.239123,3,0,-0.1,1.046,1.233678,1.139123,1.048698,2.301550,2.126114,1.004414,0.002698,0.95
4,0,4,fishing,1.292003,4,0,-0.1,1.046,1.323130,1.192003,1.030870,1.192003,1.030870,0.976475,-0.015130,0.95
5,1,0,hammock,NaN,0,0,-0.1,1.046,1.000000,0.881015,1.056596,4.721424,4.743221,0.981015,0.010596,0.95
6,1,1,hammock,NaN,0,0,-0.1,1.046,1.000000,0.921590,1.039141,3.768079,3.874017,1.021590,-0.006859,0.95
7,1,2,hammock,NaN,0,0,-0.1,1.046,1.000000,0.899238,1.052078,2.836852,2.989693,0.999238,0.006078,0.95
8,1,3,hammock,NaN,0,0,-0.1,1.046,1.000000,0.893793,1.038037,1.887530,2.031775,0.993793,-0.007963,0.95
9,1,4,hammock,NaN,0,0,-0.1,1.046,1.000000,0.912283,1.052177,0.912283,1.052177,1.012283,0.006177,0.95


## Data Moments

For the setup of the estimation we first have to choose a set of moments that we will use to match the observed data and the simulated model. For this model we include two sets of moments: 

1. The first set are Robinson's **choice frequencies** (choice frequencies here refers to the share of agents that have chosen a specific option) for each period. 
2. The second set are moments that characterize the **wage distribution** for each period, i.e. the mean of the wage of all agents that have chosen fishing in a given period and the standard deviation of the wages. 

In addition to the data, we need the complete set of potential choice options for Robinson. Respy lets us extract them from the model parameters and options.

In [7]:
optim_paras = process_params_and_options(params_true, options)[0]
choice_options_robinson = optim_paras["choices"].keys()

We need a function that computes the set of moments on the observed and simulated data.

In [8]:
def get_moments(df, choice_options):

    periods = sorted(df['Period'].unique())

    moments_dict = {}

    # 1. Choice frequencies for each period and each choice option
    info_period = df.groupby(['Period'])['Choice'].value_counts(normalize=True).to_dict()
    
    # During optimization we might have no one spending time
    # in the hammock.
    info_period = defaultdict(lambda: 0.00, info_period)

    choices_dict = dict.fromkeys(periods)
    for period in periods:
        choice_proba = []
        for choice in choice_options:
            choice_proba.append(info_period[(period, choice)])   
        choices_dict[period] = choice_proba  

    moments_dict['Choice Frequencies'] = choices_dict
    
    # 2. Wage Distribution    
    info_period = df.groupby(['Period'])['Wage'].describe() 
    wages_dict = dict.fromkeys(periods)
    for period in periods:                
        wages_dict[period] = info_period.loc[period, ['mean', 'std']].to_list()
        wages_dict[period] = [i for i in wages_dict[period] if str(i) != 'nan']
        
    # Add wage dictionaryto  moments dict.
    moments_dict['Wage Distribution'] = wages_dict
    
    return moments_dict

Now we are ready to calculate the moments.

In [9]:
moments_obs = get_moments(data_obs, choice_options_robinson)
print('Choice Frequencies')
print(moments_obs['Choice Frequencies'])
print('\n Wage Distribution')
print(moments_obs['Wage Distribution'])

Choice Frequencies
{0: [0.695, 0.305], 1: [0.695, 0.305], 2: [0.695, 0.305], 3: [0.695, 0.305], 4: [0.695, 0.305]}

 Wage Distribution
{0: [1.0031151638841938, 0.009307308674332144], 1: [1.072281554697441, 0.011004518604175825], 2: [1.1507731648868074, 0.011728243782970382], 3: [1.2327387643777055, 0.012045181928479797], 4: [1.322862242193345, 0.012440569842889777]}


## Weighting Matrix

Next we specify a weighting matrix. It needs to be a square matrix with the same number of diagonal elements as there are moments. One option would be to use the identity matrix, but we use a weighting matrix that adjusts for the variance of each moment. The variances for the moments are constructed using a bootstrapping procedure. 

In [10]:
def get_weighting_matrix(df_base, choice_options, num_boots, num_agents_smm):
    """This function constructs the weighing matrix."""
    # Ensure reproducibility
    np.random.seed(123)

    # Distribute clear baseline information.
    index_base = df_base['Identifier'].unique()
    moments_base = get_moments(df_base, choice_options)

    # Initialize counters to keep track of attempts.
    moments_sample = list()

    for _ in range(num_boots):
        sample_ids = np.random.choice(index_base, num_agents_smm, replace=False)
        moments_boot = get_moments(df_base.loc[sample_ids, :], choice_options)
        moments_sample.append(moments_boot)

    # Construct the weighing matrix based on the sampled moments.
    stats = []
    for moments_boot in moments_sample:
        stats.append(moments_dict_to_list(moments_boot))
    moments_var = np.array(stats).T.var(axis=1)

    return np.diag(moments_var ** (-1))

In [11]:
W = get_weighting_matrix(data_obs, choice_options_robinson, 300, 500)

In [12]:
pd.DataFrame(W)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,958.873438,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,0.000000,958.873438,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,0.000000,0.000000,1086.536189,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,0.000000,0.000000,0.000000,1086.536189,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,0.000000,0.000000,0.000000,0.000000,896.793382,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
5,0.000000,0.000000,0.000000,0.000000,0.000000,896.793382,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,948.297746,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,948.297746,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,930.906216,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,930.906216,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


## Criterion Function 

We have collected the observed data for our model, chosen the set of moments we want to use for estimation and defined a weighting matrix based on these moments. We can now set up the criterion function to use for estimation. 

As already discussed above, the criterion function is given by the weighted square product of the difference between observed moments $m(X)$ and simulated moments $m(\hat{X}| \theta)$. Trivially, if we have that $m(X) = m(\hat{X}| \theta)$, the criterion function returns a value of 0. Thus, the closer $\theta$ is to the real parameter vector, the smaller should be the value for the criterion function. 

In [13]:
def criterion_smm(params_cand, options, weighting_matrix, moments_obs, choice_options):
    params = params_cand.copy()
    periods = range(options["n_periods"])
    num_moments = len(np.diag(weighting_matrix))

    # Calculate simulated moments for model with candidate parameters.
    simulate = rp.get_simulate_func(params, options)
    df_sim = simulate(params)
    moments_sim = get_moments(df_sim, choice_options)
    
    # Check whether candidate parameters are valid inputs.
    stats_obs, stats_sim = [], []
    for group in moments_sim.keys():
        for period in periods:
            stats_obs.extend(moments_obs[group][(period)])
            stats_sim.extend(moments_sim[group][(period)])
            
    is_valid = len(stats_obs) == len(stats_sim) == num_moments
    
    # Calculate weigthed dot product of difference between real moments and 
    # simulated moments.
    if is_valid:
        moments_error = np.array(stats_obs) - np.array(stats_sim)
        fval = moments_error @ weighting_matrix @ moments_error
    else:
        fval = 1000000
        
    return fval

Criterion function at the true parameter vector:

In [14]:
fval = criterion_smm(params_true, options, W, moments_obs, choice_options_robinson)
fval

0.0

We can plot the criterion function to examine its behavior around the minimum in more detail. The plots below show the criterion function at varying values of all parameters in the the paramter vector.

In [ ]:
criterion_args = [options, W, moments_obs, choice_options_robinson]
plot_criterion_params(params_true, criterion_args, criterion_smm)

This depiction conceals the fact that the criterion function is not a smooth function of our parameter values. We can reveal this property if we 'zoom into' the function far enough. The plots below show the criterion function for varying values of *delta* around the true minimum value of 0.95. We can see that the function exhibits small plateaus and is thus not completely smooth. 

In [ ]:
criterion_args = [options, W, moments_obs, choice_options_robinson]
plot_criterion_detail(params_true, criterion_args, criterion_smm)

##  Estimation Exercise

In the following we will conduct a simulation exercise to estimate the parameter vector using our criterion function and weighting matrix. We will begin by simulating data using the new parameter vector and examine how the simulated moments differ from the observed ones. We will then use an optimizer to minimize the criterion function in order to retrieve the true parameter vector. Additionally, we will explore how the criterion function behaves if we change the simulation seed or misspecify the constraints by fixing parameters at the wrong values.

### Estimation for one parameter

For now, our candidate parameter vector will just differ in *delta* from the true parameters.

In [ ]:
# Define candidate parameter vector

#### Simulated Moments

We can now use our model to simulate data using the candidate parameter vector. We can see that the choice frequencies and wage distribution differ from the moments of the observed dataset.

In [ ]:
# Simulate data and compute simulated moments

We can plot the moments to compare the choice frequencies for each period.

In [ ]:
plot_moments_choices(moments_obs, moments_sim)

The plots below show the mean and the standard deviation in the wages for each period.

In [ ]:
plot_moments_wage(moments_obs, moments_sim)

The criterion function value for the candidate parameter vector is not zero.

In [ ]:
# Criterion function for candidate parameter vector

#### Optimization Procedure

We will now use an optimization procedure to retrieve the true parameter vector. For the optimization we can use [estimagic](https://estimagic.readthedocs.io/en/latest/index.html). In order to minimize the criterion function we need estimagic's `minimize` function.



In [ ]:
from estimagic.optimization.optimize import minimize

We have verified above that the criterion function gives a value of 0 for the true parameter vector. Before we try different parameter specifications, we can check whether an optimizer recognizes the true vector as the minimum of our criterion function.

In [ ]:
# Save criterion function arguments in a dictionary to pass onto the optimzer.
criterion_kwargs = dict()
criterion_kwargs["weighting_matrix"] = W
criterion_kwargs["options"] = options
criterion_kwargs["choice_options"] = list(choice_options_robinson)
criterion_kwargs["moments_obs"] = moments_obs

As the code below shows, the optimization algorithm recognizes the true parameter vector as the minimum of the criterion function as it returns a function value of 0 and the true parameter values.

In [ ]:
# Estimation with true parameter vector

##### Upper and Lower Bounds

We can help the optimizer by specifying bounds for the parameters. Since we know the true parameters in the case of this model, we can just pick upper and lower bounds that are fairly close to the true values of the parameters to aid the optimizer in the search for the optimum. By default, the upper and lower bounds are set to $\infty$ and $-\infty$, so specifying upper and lower bounds substantially reduces the range of parameter values that the optimizer can potentially cover.

For optimization with estimagic, we can specify bounds by adding the columns *'lower'* and *'upper'* to the dataframe that contains the parameter values.

In [ ]:
params_cand['lower'] = [0.89, 0.066, -0.11, 1.04, 0, 0, 0]
params_cand['upper'] = [0.98, 0.072, -0.095, 1.055, 0.1, 0.1, 0.1]
params_cand

##### Constraints

Additionally we hold all other parameters fixed for now to aid the optimizer in finding the optimal value for *delta*.

In [ ]:
# Define base constraints to use for the rest of the notebook.
constr_base = [
    {"loc": "shocks_sdcorr", "type": "sdcorr"}, 
    {"loc": "delta", "type": "fixed"},
    {"loc": "wage_fishing", "type": "fixed"},
    {"loc": "nonpec_fishing", "type": "fixed"},
    {"loc": "nonpec_hammock", "type":"fixed"},
    {"loc": "shocks_sdcorr", "type": "fixed"},
]

In [ ]:
# Remove constraint for delta.

##### Optimize

We can now optimize the criterion function with respect to the parameter vector. The optimizer manages to reach a function value of 0 and finds an approximately correct *delta* for our model. 

This exercise again reveals that we are dealing with a non-smooth criterion function. The optimizer does not return the exact value of 0.95 for *delta* because of the little plateaus we saw when zooming into the criterion function. As the plot shows, there is a small area around the true value for *delta* that also returns a function value of 0 and might thus be picked by the optimizer.

In [ ]:
rslt = minimize(criterion=criterion_smm,
                params=params_cand,
                algorithm="nlopt_bobyqa",
                constraints=constr,
                criterion_kwargs=criterion_kwargs,
               )
rslt[0]

### Chatter in the Criterion Function

In this exercise we explore the sensitivity of the criterion function to the choice of simulation seed and number of agents.

#### Changing the Simulation Seed

As shown above, the optimizer manages to find a function value of exactly 0 for the true parameter vector. This is the case because respy controls the realization of random elements in the simulation process via a simulation seed. The model thus always simulates the exact same dataset for a given parameter vector. Our criterion function becomes exactly 0 at the true parameter vector because for this vector, the observed and simulated data are identical.

Changing the simulation seed results in simulated moments that, even for the true parameter vector, are never completely equal to the observed moments. 

Let's estimate the model with a different simulation seed.

In [ ]:
# Change seed

We can see that the criterion function is not exactly 0 anymore for the true parameter vector.

In [ ]:
# Criterion function with new seed

Our optimizer thus also does not return a function value of 0 for the true parameter vector.

In [ ]:
# Change criterion_kwargs

In [ ]:
# Estimation using true parameter vector

Since the optimizer does not even recognize the true parameter vector, it is also not able to reach a criterion function  value of 0 for a different parameter vector.

In [ ]:
# Estimation using candidate parameter vector

#### Multiple Simulation Seeds

The section above shows what happens to the criterion function if we change the seed for simulating the data. In the sections below we will repeat this exercise for multiple seeds and explore what happens if we increase the number of simulated as well as observed agents in our model.

In [ ]:
# List of seeds, includes true simulation seed of 132.

In [ ]:
# Define other inputs for criterion function
criterion_args = copy.deepcopy(criterion_kwargs)
criterion_args['params'] = params_true.copy()

The plot below shows the criterion function for different simulation seeds, including the true one. We can seed that different seeds lead to different fits between the simulated and observed data, with some seeds performing worse than others. Only the true seed leads to a function value of 0.

In [ ]:
plot_chatter(seeds, criterion_smm, criterion_args)

#### Changing the Simulation Seed for increasing Sample Size of Simulated Agents

We now repeat this exercise for an increasing number of simulated agents. As the plot below shows, increasing the number of simulated agents reduces the chatter considerably but the criterion function remains consistently above zero. Only the simulated sample of 1000 agents at the true simulation seed reaches a function of zero, since, as discussed before, the simulated sample is identical to the observed one for this calibration.

In [ ]:
# Changing the number of agents.

In [ ]:
plot_chatter_numagents_sim(seeds, num_agents, criterion_smm, criterion_args)

#### Changing the Simulation Seed for increasing Sample Size of Simulated *and* Observed Agents

We now increase not only the number of simulated agents but also the number of observed agents in our sample. Doing so decreases the chatter in the criterion function as before but also levels out the function around 0 for all simulation seeds. For large enough samples of observed and simulated agents the choice of simulation seed is thus irrelevant.

In [ ]:
plot_chatter_numagents_both(seeds, num_agents, get_moments, criterion_smm, criterion_args)

*Note*: In contrast to the previous plot, we can see that the function reaches a value of 0 at the true simulation seed for all quantities of agents. This is the case because we now simultaneously increase the number of both simulated and observed agents while the observed sample remained untouched in the previous plot. Increasing the number of agents simultaneously for both groups creates identical samples for the true simulation seed at each quantity of agents. 

### Moving away from the Optimum

For the next exercise we explore what happens to the criterion function if we move away from the optimum. We do this in a controlled fashion by changing the parameter values for *delta* and *wage_fishing* and fixing the latter in the constraints. Doing so should lead *delta* to move away from its optimal value of 0.95 during optimization.

In [ ]:
# Change value for wage_fishing

The optimizer cannot retrieve the true parameter and does not reach a value of 0.

In [ ]:
rslt_wrong_fix = minimize(criterion=criterion_smm,
                params=params_cand,
                algorithm="nlopt_bobyqa",
                constraints=constr,
                criterion_kwargs=criterion_kwargs,
               )
rslt_wrong_fix[0]

#### Retrieving the true Parameter Vector

We now repeat the estimation with the new parameter vector and free up *wage_fishing* to retrieve the optimal values for both parameters.

The parameter for *wage_fishing* is still 0.072 since we fixed it for the prior estimation:

In [ ]:
# Copy parameters from last step

We now free up *wage_fishing* in the constraints in addition to *delta*.

In [ ]:
# Adjust constraints to free up both delta and wage_fishing.

Freeing up the non-optimal *wage_fishing* improves the estimates. The criterion function value is much closer to 0 and the optimizer manages to retrieve the true parameter values quite closely.

In [ ]:
rslt_unfix = minimize(criterion=criterion_smm,
                params=params_cand,
                algorithm="nlopt_bobyqa",
                constraints=constr_u,
                criterion_kwargs=criterion_kwargs,
               )
rslt_unfix[0]

For easier comparison, we can compute the difference between the true and estimated value:

In [ ]:
deviation = params_true['value'] - rslt_unfix[1]['value']
deviation

### Derivative-Based Optimization Algorithm

So far we have used only one optimization algorithm to estimate the parameter vector. The algorithm we used, BOBYQA (Bound Optimization by Quadratic Approximation), is a derivative-free optimization algorithm which works fairly well on our criterion function. As discussed above, our criterion function is a step function that contains plateaus for certain ranges of parameter values. An important implication of this property is that we cannot calculate proper derivatives and thus derivative-based optimization algorithms will not work to estimate the parameters. 

To demonstrate this problem, we will now try to estimate the parameters using an optimization algorithm that does use derivatives during optimization.

In [ ]:
# Define candidate parameter vector and constraints.
params_cand = params_true.copy()
params_cand.loc['delta','value'] = 0.93

params_cand['lower'] = [0.89, 0.066, -0.11, 1.04, 0, 0, 0]
params_cand['upper'] = [0.98, 0.072, -0.095, 1.055, 0.1, 0.1, 0.1]

constr = constr_base.copy()
constr.remove({"loc": "delta", "type": "fixed"})

We try the L-BFGS-B (Limited-Memory-Broyden–Fletcher–Goldfarb–Shanno with box constraints) algorithm. As shown below, L-BFGS-B fails and the optimizer returns the same parameter vector that we used as an input. 

In [ ]:
rslt = minimize(criterion=criterion_smm,
                params=params_cand,
                algorithm="scipy_L-BFGS-B",
                constraints=constr,
                criterion_kwargs=criterion_kwargs,
               )
rslt[0]

## References

* Adda, J., & Cooper, R. W. (2003). *Dynamic Economics: Quantitative Methods and Applications*. MIT press.


* Adda, J., Dustmann, C., & Stevens, K. (2017). The Career Costs of Children. *Journal of Political Economy*, 125(2), 293-337.


* Andrews, I., Gentzkow, M., & Shapiro, J. M. (2017). Measuring the Sensitivity of Parameter Estimates to Estimation Moments. *The Quarterly Journal of Economics*, 132(4), 1553-1592.


* Bruins, M., Duffy, J. A., Keane, M. P., & Smith Jr, A. A. (2018). Generalized Indirect Inference for Discrete Choice Models. *Journal of econometrics*, 205(1), 177-203.


* Davidson, R., & MacKinnon, J. G. (2004). *Econometric Theory and Methods (Vol. 5)*. New York: Oxford University Press.


* Evans, R. W. (2018, July 5). Simulated Method of Moments (SMM) Estimation. Retrieved November 30, 2019, from https://notes.quantecon.org/submission/5b3db2ceb9eab00015b89f93.


* Frazier, D. T., Oka, T., & Zhu, D. (2019). Indirect Inference with a Non-Smooth Criterion Function. *Journal of Econometrics*, 212(2), 623-645.
 
 
* Gourieroux, M., & Monfort, D. A. (1996). *Simulation-based econometric methods*. Oxford university press.


* McFadden, D. (1989). A Method of Simulated Moments for Estimation of Discrete Response Models without Numerical Integration. *Econometrica: Journal of the Econometric Society*, 995-1026.